In [1]:
#Importamos las bibliotecas a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
#Leemos los archivos a trabajar
df1 = pd.DataFrame( pd.read_csv("meteo-nogal-09.csv", engine="python" ) )
df2 = pd.DataFrame( pd.read_csv("soil-nogal-09.csv"))

In [3]:
df1.head()

,DATE,TIME,Unnamed: 2,Unnamed: 3,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,TS,NaN,NaN,NaN,m/s,m/s,m/s,degC,mV,w/m2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-09,00:10,1.0,NaN,1.215,0.506,-0.044,10.95,1.159,-176.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-09,00:20,1.0,NaN,0.892,0.588,-0.072,10.81,1.159,-176.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-09,00:30,1.0,NaN,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-09,00:40,1.0,NaN,1.135,0.7410001,-0.093,9.8,1.258,-182.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df1.dtypes

DATE            object
TIME            object
Unnamed: 2     float64
Unnamed: 3     float64
u_Avg           object
v_Avg           object
w_Avg           object
t_Avg           object
kh20_Avg        object
net_rad_Avg     object
shf1_Avg        object
shf2_Avg        object
vv_Avg          object
airT_Avg        object
rh_Avg         float64
e_sat_Avg       object
e_Avg           object
h2o_hmp_Avg     object
Unnamed: 18    float64
Unnamed: 19     object
Unnamed: 20     object
Unnamed: 21    float64
Unnamed: 22     object
Unnamed: 23     object
Unnamed: 24     object
Unnamed: 25     object
Unnamed: 26     object
Unnamed: 27     object
Unnamed: 28     object
Unnamed: 29     object
Unnamed: 30     object
Unnamed: 31     object
Unnamed: 32     object
Unnamed: 33     object
Unnamed: 34     object
Unnamed: 35     object
dtype: object

In [5]:
df2.dtypes

id                        int64
2 Year_RTM  L             int64
3 Day_RTM  L              int64
4 Hour_Minute_RTM  L      int64
5 T108v_1_AVG  L        float64
6 T108v_2_AVG  L        float64
7 T108v_3_AVG  L        float64
8 T108v_4_AVG  L        float64
9 T108v_5_AVG  L        float64
10 T108v_6_AVG  L       float64
11 T108v_7_AVG  L       float64
12 T108v_8_AVG  L       float64
Tsuelo_10cm             float64
Tsuelo_20cm             float64
Tsuelo_30cm             float64
Tsuelo_40cm             float64
Tsuelo_55cm             float64
Tsuelo_70cm             float64
Tsuelo_85cm             float64
Tsuelo_100cm            float64
21 Tref_AVG  L          float64
22 period_1  L          float64
23 period_2  L          float64
24 period_3  L          float64
25 period_4  L          float64
26 period_5  L          float64
27 period_6  L          float64
28 period_7  L          float64
29 period_8  L          float64
30 TVWC_1  L            float64
                         ...   
35 TVWC_

In [6]:
#Es necesario adaptar los DataFrames para poder trabajar con ellos.
#Eliminamos la fila que dicta las unidades de cada columna en el df1
df1 = df1.drop(df1.columns[df1.columns.str.contains('unnamed:',case=False)],axis=1)
df1 = df1.drop(df1.index[0]).reset_index(drop=True)

In [7]:
#Vemos que el df2 tiene sus unidades de tiempo cada media hora. Necesitamos que los tiempos del df1 sean de la 
#misma manera.
#Creamos una variable DATETIME que junte las columnas DATE y TIME para df1
df1["DATETIME"] = df1["DATE"] +" "+ df1["TIME"]
#Eliminamos las columnas DATE y TIME
df1.drop( ["DATE","TIME"], axis=1, inplace=True )

In [8]:
df1.head()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,DATETIME
0,1.215,0.506,-0.044,10.95,1.159,-176.5,23.58,-26.72,1.327,9.4,0.899,1.178,1.059,8.12,01-01-09 00:10
1,0.892,0.588,-0.072,10.81,1.159,-176.6,23.73,-26.76,1.079,9.24,0.896,1.166,1.045,8.02,01-01-09 00:20
2,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,23.61,-26.46,1.421,9.059999,0.896,1.152,1.032,7.926001,01-01-09 00:30
3,1.135,0.7410001,-0.093,9.8,1.258,-182.7,23.41,-26.46,1.384,9.059999,0.874,1.152,1.008,7.737,01-01-09 00:40
4,0.7550001,1.064,-0.149,8.87,1.332,-184.8,23.22,-26.39,1.359,8.63,0.859,1.119,0.961,7.39,01-01-09 00:50


In [9]:
#Convertimos nuestra variable DATETIME en una variable FECHA del tipo date
df1['FECHA'] = pd.to_datetime(df1.apply(lambda x: x['DATETIME'], 1), dayfirst=True)
df1 = df1.drop(['DATETIME'], 1)
df1.head(3)

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA
0,1.215,0.506,-0.044,10.95,1.159,-176.5,23.58,-26.72,1.327,9.4,0.899,1.178,1.059,8.12,2009-01-01 00:10:00
1,0.892,0.588,-0.072,10.81,1.159,-176.6,23.73,-26.76,1.079,9.24,0.896,1.166,1.045,8.02,2009-01-01 00:20:00
2,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,23.61,-26.46,1.421,9.059999,0.896,1.152,1.032,7.926001,2009-01-01 00:30:00


In [10]:
#De nuestra variable FECHA, obtenemos una nueva variable que contabilice los minutos transcurridos
df1['MIN'] = df1['FECHA'].dt.minute
df1.head(6)

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
0,1.215,0.506,-0.044,10.95,1.159,-176.5,23.58,-26.72,1.327,9.4,0.899,1.178,1.059,8.12,2009-01-01 00:10:00,10.0
1,0.892,0.588,-0.072,10.81,1.159,-176.6,23.73,-26.76,1.079,9.24,0.896,1.166,1.045,8.02,2009-01-01 00:20:00,20.0
2,1.227,0.6850001,-0.06900001,10.34,1.224,-175.5,23.61,-26.46,1.421,9.059999,0.896,1.152,1.032,7.926001,2009-01-01 00:30:00,30.0
3,1.135,0.7410001,-0.093,9.8,1.258,-182.7,23.41,-26.46,1.384,9.059999,0.874,1.152,1.008,7.737,2009-01-01 00:40:00,40.0
4,0.7550001,1.064,-0.149,8.87,1.332,-184.8,23.22,-26.39,1.359,8.63,0.859,1.119,0.961,7.39,2009-01-01 00:50:00,50.0
5,0.8850001,0.992,-0.167,8.76,1.45,-183.4,23.33,-26.98,1.39,8.34,0.840,1.097,0.9210001,7.089,2009-01-01 01:00:00,0.0


In [11]:
#Filtramos ahora las filas de nuestro df1 para los valores de cada media hora, es decir, aquellos donde la variable
#'MIN' es igual a 30.0 (media hora) o a 0.0 (una hora); filtramos también todos aquellos que no sean del año 2009.
df1 = df1[((df1["MIN"] == 30.0) | (df1["MIN"] == 0.0)) & (df1["FECHA"] < '2010-01-01 00:30:00')]
df1 = df1.reset_index(drop=True)
df1.tail()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
17441,1.676,-0.139,14.36,1113,-79.21,NAN,NAN,3.067,14.7,0.407,1.672,0.681,5.126,NaN,2009-12-31 22:00:00,0.0
17442,1.196,-0.12,14.17,1170,-78.89,NAN,NAN,2.683,14.63,0.388,1.664,0.645,4.855,NaN,2009-12-31 22:30:00,30.0
17443,0.454,-0.074,12.63,1060,-72.47,NAN,NAN,2.419,13,0.472,1.497,0.707,5.354,NaN,2009-12-31 23:00:00,0.0
17444,0.746,-0.116,11.9,1043,-74.88,NAN,NAN,2.38,12.3,0.5,1.430,0.715,5.428,NaN,2009-12-31 23:30:00,30.0
17445,0.72,-0.078,11.02,1071,-70.44,NAN,NAN,2.068,11.28,0.521,1.336,0.696,5.299,NaN,2010-01-01 00:00:00,0.0


In [12]:
#Para nuestro df2, necesitamos acomodar los minutos de igual manera. Para esto, necesitamos separar las horas de manera
#que el tiempo se denote "HORA:MINUTOS".
#Convertimos la variable '4 Hour_Minute_RTM  L' y '2 Year_RTM  L ' a object
df2['4 Hour_Minute_RTM  L'] = df2['4 Hour_Minute_RTM  L'].astype(str)
df2['2 Year_RTM  L'] = df2['2 Year_RTM  L'].astype(str)

In [13]:
#Reescribimos cada hora como el formato deseado
x = [] #Arreglo que almacena los valores de cada hora en el formato adecuado.
for i in range(0,len(df2)):
    if (len(df2['4 Hour_Minute_RTM  L'][i])==2):
        x.append('00' + ':' + str(df2['4 Hour_Minute_RTM  L'][i])[0:]+ ':' + '00')
    elif (len(df2['4 Hour_Minute_RTM  L'][i])==3):
        x.append('0' + str(df2['4 Hour_Minute_RTM  L'][i])[:1] + ':' + str(df2['4 Hour_Minute_RTM  L'][i])[1:] +':'+'00')
    elif ((len(df2['4 Hour_Minute_RTM  L'][i])==4) & ((df2['4 Hour_Minute_RTM  L'][i])!='2400')):
        x.append(str(df2['4 Hour_Minute_RTM  L'][i])[:2] + ':' + str(df2['4 Hour_Minute_RTM  L'][i])[2:]+':'+'00')
    elif ((len(df2['4 Hour_Minute_RTM  L'][i])==4) & ((df2['4 Hour_Minute_RTM  L'][i])=='2400')):
        x.append('00'+':'+'00'+':'+'00')

In [14]:
#Asignamos nuestro arreglo de horas como una nueva variable 'TIEMPO'
df2['TIEMPO'] = x
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_30cm.1,THETA_40cm.1,THETA_55cm.1,THETA_70cm.1,THETA_85cm.1,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:30:00
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:00:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23:30:00
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:00:00


In [15]:
#Necesitamos ahora una variable que indique el mes del año y el día del mes en el que se tomo el dato. 
#Para esto, recurrimos de manera rudimentaria a un calendario del año 2009 e indicamos al programa qué días abarca 
#cada mes:
y = [] #Arreglo que almacena el valor de mes para cada día.
z = [] #Arreglo que almacena el día del mes .
for i in range (0,len(df2)):
    d = df2['3 Day_RTM  L'][i]
    if (1 <= df2['3 Day_RTM  L'][i] <= 31):
        y.append(1)
        z.append(d)
    elif(32 <= df2['3 Day_RTM  L'][i] <= 59):
        d = d-31
        y.append(2)
        z.append(d)
    elif(60 <= df2['3 Day_RTM  L'][i] <= 90):
        d = d-59
        y.append(3)
        z.append(d)
    elif(91 <= df2['3 Day_RTM  L'][i] <= 120):
        d = d - 90
        y.append(4)
        z.append(d)
    elif(121 <= df2['3 Day_RTM  L'][i] <= 151):
        d = d - 120
        y.append(5)
        z.append(d)
    elif(152 <= df2['3 Day_RTM  L'][i] <= 181):
        d = d - 151
        y.append(6)
        z.append(d)
    elif(181 <= df2['3 Day_RTM  L'][i] <= 212):
        d = d -181
        y.append(7)
        z.append(d)
    elif(213 <= df2['3 Day_RTM  L'][i] <= 243):
        d = d - 212
        y.append(8)
        z.append(d)
    elif(244 <= df2['3 Day_RTM  L'][i] <= 273):
        d = d-243
        y.append(9)
        z.append(d)
    elif(274 <= df2['3 Day_RTM  L'][i] <= 304):
        d = d-273
        y.append(10)
        z.append(d)
    elif(304 <= df2['3 Day_RTM  L'][i] <= 334):
        d = d-304
        y.append(11)
        z.append(d)
    elif(335 <= df2['3 Day_RTM  L'][i] <= 365):
        d = d - 334
        y.append(12)
        z.append(d)

In [16]:
#Asignamos la nueva variable 'MES' a nuestro DataFrame df2
df2['MES'] = y
df2['DIA'] = z

In [17]:
#Convertimos las variables 'MES' y día en tipo object
df2['MES'] = df2['MES'].astype(str)
df2['DIA'] = df2['DIA'].astype(str)

In [18]:
#Reescribimos la variable 'MES' al formato deseado
y = []
for i in range (0,len(df2)):
    if(len(df2['MES'][i])==1):
        y.append("0" + df2['MES'][i])
    elif(len(df2['MES'][i])==2):
        y.append(df2['MES'][i])

In [19]:
#Introducimos una variable AÑO para el formato deseado
w = []
for i in range (0,len(df2)):
    i
    w.append("09")

In [20]:
#Reestablecemos nuestras variable
df2['MES'] = y
df2['DIA'] = z
df2['AÑO'] = w

In [21]:
df2.dtypes

id                        int64
2 Year_RTM  L            object
3 Day_RTM  L              int64
4 Hour_Minute_RTM  L     object
5 T108v_1_AVG  L        float64
6 T108v_2_AVG  L        float64
7 T108v_3_AVG  L        float64
8 T108v_4_AVG  L        float64
9 T108v_5_AVG  L        float64
10 T108v_6_AVG  L       float64
11 T108v_7_AVG  L       float64
12 T108v_8_AVG  L       float64
Tsuelo_10cm             float64
Tsuelo_20cm             float64
Tsuelo_30cm             float64
Tsuelo_40cm             float64
Tsuelo_55cm             float64
Tsuelo_70cm             float64
Tsuelo_85cm             float64
Tsuelo_100cm            float64
21 Tref_AVG  L          float64
22 period_1  L          float64
23 period_2  L          float64
24 period_3  L          float64
25 period_4  L          float64
26 period_5  L          float64
27 period_6  L          float64
28 period_7  L          float64
29 period_8  L          float64
30 TVWC_1  L            float64
                         ...   
39 TVWCF

In [22]:
#Convertimos las variables 'MES' y día en tipo object
df2['MES'] = df2['MES'].astype(str)
df2['DIA'] = df2['DIA'].astype(str)

In [23]:
#Creamos una variable DATE que junte las columnas AÑO, MES y DÍA para df2
df2['DATE'] = df2['DIA']  + "-" + df2['MES'] + "-" + df2['AÑO']
#Eliminamos las columnas AÑO, MES y DIA
#df2.drop( ['AÑO','MES','DIA'], axis=1, inplace=True )

In [24]:
#Creamos una variable DATETIEMPO que junte las columnas DATE y TEMPO para df2
df2['DATETIEMPO'] = df2['DATE'] +" "+ df2['TIEMPO']
#Eliminamos las columnas DATE y TIME
#df2.drop( ['DATE','TIEMPO'], axis=1, inplace=True )

In [25]:
#Corregimos la última fecha para la variable DATETIEMPO
t = []
j = 0
for i in range(0,len(df2)):
    j = j + 1
    if(j<len(df2)):
        t.append(df2['DATETIEMPO'][i])
    if(j==len(df2)):
        t.append('01-01-10 00:00:00')

In [26]:
#Reestablecemos la variable DATETIEMPO
df2['DATETIEMPO'] = t

In [27]:
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,THETA_100cm.1,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO,MES,DIA,AÑO,DATE,DATETIEMPO
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,NaN,22:00:00,12,31,09,31-12-09,31-12-09 22:00:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,22:30:00,12,31,09,31-12-09,31-12-09 22:30:00
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,23:00:00,12,31,09,31-12-09,31-12-09 23:00:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,23:30:00,12,31,09,31-12-09,31-12-09 23:30:00
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,NaN,00:00:00,12,31,09,31-12-09,01-01-10 00:00:00


In [28]:
#Convertimos nuestra variable DATETIEMPO en una variable FECHA del tipo date, al igual a como se hizo para df1
df2['FECHA'] = pd.to_datetime(df2.apply(lambda x: x['DATETIEMPO'], 1), dayfirst=True)
#df2 = df2.drop(['DATETIEMPO'], 1)
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO,MES,DIA,AÑO,DATE,DATETIEMPO,FECHA
17477,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,22:00:00,12,31,09,31-12-09,31-12-09 22:00:00,2009-12-31 22:00:00
17478,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,22:30:00,12,31,09,31-12-09,31-12-09 22:30:00,2009-12-31 22:30:00
17479,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,23:00:00,12,31,09,31-12-09,31-12-09 23:00:00,2009-12-31 23:00:00
17480,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,23:30:00,12,31,09,31-12-09,31-12-09 23:30:00,2009-12-31 23:30:00
17481,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,00:00:00,12,31,09,31-12-09,01-01-10 00:00:00,2010-01-01 00:00:00


In [29]:
#Quitamos ahora las fechas duplicadas de nuestro segundo DataFrame
df2 = pd.DataFrame(df2.drop_duplicates(['FECHA']))
df2 = df2.reset_index(drop=True)
df2.tail()

,id,2 Year_RTM L,3 Day_RTM L,4 Hour_Minute_RTM L,5 T108v_1_AVG L,6 T108v_2_AVG L,7 T108v_3_AVG L,8 T108v_4_AVG L,9 T108v_5_AVG L,10 T108v_6_AVG L,...,62 batt L.1,63 lluvia_TOT L,Rs (splite).1,TIEMPO,MES,DIA,AÑO,DATE,DATETIEMPO,FECHA
17421,148,2009,365,2200,0.944,0.950,0.977,0.992,1.012,1.045,...,NaN,NaN,NaN,22:00:00,12,31,09,31-12-09,31-12-09 22:00:00,2009-12-31 22:00:00
17422,148,2009,365,2230,0.944,0.951,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,22:30:00,12,31,09,31-12-09,31-12-09 22:30:00,2009-12-31 22:30:00
17423,148,2009,365,2300,0.944,0.952,0.977,0.993,1.012,1.045,...,NaN,NaN,NaN,23:00:00,12,31,09,31-12-09,31-12-09 23:00:00,2009-12-31 23:00:00
17424,148,2009,365,2330,0.944,0.952,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,23:30:00,12,31,09,31-12-09,31-12-09 23:30:00,2009-12-31 23:30:00
17425,148,2009,365,2400,0.943,0.953,0.978,0.993,1.012,1.045,...,NaN,NaN,NaN,00:00:00,12,31,09,31-12-09,01-01-10 00:00:00,2010-01-01 00:00:00


In [60]:
#Quitamos ahora las fechas duplicadas de nuestro primer DataFrame
df1 = pd.DataFrame(df1.drop_duplicates(['FECHA']))
df1 = df1.reset_index(drop=True)
df1.tail()

,u_Avg,v_Avg,w_Avg,t_Avg,kh20_Avg,net_rad_Avg,shf1_Avg,shf2_Avg,vv_Avg,airT_Avg,rh_Avg,e_sat_Avg,e_Avg,h2o_hmp_Avg,FECHA,MIN
17387,1.676,-0.139,14.36,1113,-79.21,NAN,NAN,3.067,14.7,0.407,1.672,0.681,5.126,NaN,2009-12-31 22:00:00,0.0
17388,1.196,-0.12,14.17,1170,-78.89,NAN,NAN,2.683,14.63,0.388,1.664,0.645,4.855,NaN,2009-12-31 22:30:00,30.0
17389,0.454,-0.074,12.63,1060,-72.47,NAN,NAN,2.419,13,0.472,1.497,0.707,5.354,NaN,2009-12-31 23:00:00,0.0
17390,0.746,-0.116,11.9,1043,-74.88,NAN,NAN,2.38,12.3,0.5,1.430,0.715,5.428,NaN,2009-12-31 23:30:00,30.0
17391,0.72,-0.078,11.02,1071,-70.44,NAN,NAN,2.068,11.28,0.521,1.336,0.696,5.299,NaN,2010-01-01 00:00:00,0.0


In [80]:
#Buscamos ahora las diferencias de Fecha entre nuestro primer y segundo DataFrame
#Creamos un DataFrame que mezcle ambos DataFRames
df = pd.concat([df1,df2],sort=True)
df = df.reset_index(drop=True)

In [81]:
df_gpby = df.groupby(list(df.columns))

In [ ]:
idx = [x[0] for x in df_gpby.groups.values() if len(x)==1]

In [ ]:
df.reindex(idx)

In [ ]:
#Quitamos ahora las fechas duplicadas de nuestro tercer DataFrame
df2 = pd.DataFrame(df2.drop_duplicates(['FECHA']))
df2 = df2.reset_index(drop=True)
df2.tail()